# 웹에서 파일(또는 이미지) 다운로드 및 Request Headers 수정 예제
_본 자료는 안수찬 강사님의 파이썬을 활용한 업무자동화 Camp (fast campus)의 강의자료를 기반으로 만들어졌습니다._  
만든이 : 김보섭  

## Jupyter 개발환경 구축 (tip)
* Python Shell (python)
* Interactive Shell -> IPython (ipython)
* IPython Notebook (=> Python 만을 실행할 수 있는 Notebook)
* Jupyter Notebook  

Anaconda로 환경 구성안할 시, cmd에서 아래의 command로 ipython 설치  
* install ipython : pip install ipython  
* install ipython notebook : pip install ipython[notebook]
    * localhost:8888 (==127.0.0.1)

## 웹에서 파일(또는 이미지) 다운로드

In [1]:
#fp = open('____', '___')
# 네이버에 있는 모든 이미지를 다운로드
# .jpg, .png, .svg, ....(?)
image_url = 'http://cdn.www.fastcampus.co.kr/wp-content/uploads/2016/01/fastcampus_logo_345x76.png'

In [2]:
image_url.split('/')

['http:',
 '',
 'cdn.www.fastcampus.co.kr',
 'wp-content',
 'uploads',
 '2016',
 '01',
 'fastcampus_logo_345x76.png']

In [3]:
full_filename = image_url.split('/')[-1]

In [4]:
filename = full_filename.split('.')[0]
file_extension = full_filename.split('.')[1]
print(filename, file_extension)

fastcampus_logo_345x76 png


In [5]:
import os, sys
import shutil # High-level File Operator
import requests
import re
from bs4 import BeautifulSoup

In [6]:
# 파일 (이미지, 문서, 압축파일, ...) 다운로드
response = requests.get(image_url, stream = True)
with open(full_filename, 'wb') as fp:
    shutil.copyfileobj(response.raw, fp)

In [7]:
# 블로그 검색결과의 1번 페이지 (10개 포스트) = 썸네일 이미지를 저장하는 코드
# 파이썬/
# 노드/
# 업무 자동화/
print(os.listdir())
keywords = ['파이썬', '노드', '업무 자동화']

[' Download raw contents and modify Request Headers.ipynb', '.ipynb_checkpoints', 'fastcampus_logo_345x76.png', 'os_shutil.ipynb', 'Python_basic1.ipynb', 'Python_basic2.ipynb', 'Python_basic3.ipynb', 'Scrapping static webpage.ipynb', 'Scrapping text mining papers in arXiv.py', 'Selenium.ipynb', 'Static webpage and Dynamic webpage.ipynb', '파이썬']


In [8]:
def crawl_naver_posts_by(keyword):
    print(keyword + ' 크롤링을 시작합니다.')
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' + keyword
    response = requests.get(url)
    dom = BeautifulSoup(response.text, 'html.parser')
    post_elements = dom.select('li.sh_blog_top')
    
    # 크롤링 keyword에 대하여 기본 directory 내에 keyword를 이름으로 가지는 폴더 생성
    if keyword in os.listdir():
        shutil.rmtree(keyword)
    os.mkdir(keyword)

    for post_element in post_elements:
        title = post_element.select_one('a.sh_blog_title').attrs.get('title') # 해당 블로그 post의 제목을 저장
        title = re.sub('\\W', '', title)
        image_url = post_element.select_one('img.sh_blog_thumbnail').attrs.get('src') # image의 링크를 저장
        image_response = requests.get(image_url, stream = True) # image_url에서 image를 받아서 image 형태로 get (stream = True)
        filepath = './{keyword}/{title}.jpg'.format(keyword = keyword, title = title) 
        with open(filepath, 'wb') as fp:
            shutil.copyfileobj(image_response.raw, fp)

In [9]:
crawl_naver_posts_by('파이썬')

파이썬 크롤링을 시작합니다.


In [10]:
os.listdir('./파이썬')

['코딩파이썬Python001파이썬이뭐지.jpg',
 '파이썬PythonGUI프로그램PyQt5개발시작하기.jpg',
 '파이썬기초5편입력출력조건문.jpg',
 '파이썬으로MQTTPublishSbuscribeClient구현하기.jpg',
 '파이썬학원기초교육받고SW공부시작하기.jpg']

## Request Headers 수정예제
### 다음지도 예제
다음 지도 (http://map.daum.net) 에 스타벅스를 검색했을 때, 지점들에 관한 정보를 가져오는 예제  
(Host와 Referer를 정해줘야함)

In [11]:
### 1. Host, Referer
# 2. Parsing
# request headerf를 수정
headers = {
    'Host' : 'map.search.daum.net',
    'Referer' : 'http://map.daum.net/'
    
}
url = 'http://map.search.daum.net/mapsearch/map.daum?callback=jQuery18105775289722381518_1498028927868&q=%EC%8A%A4%ED%83%80%EB%B2%85%EC%8A%A4&msFlag=A&sort=0'
response = requests.get(url, headers = headers)

In [12]:
# 1. '('로 split => 첫번째를 제외한 모든 애들 붙여서 text
# 2. ')'로 split => 맨 마지막을 제외한 모든 애들 붙여서 text
tmp = '('.join(response.text.split('(')[1:])
tmp = ''.join(response.text.split('(')[1:])
tmp = ''.join(tmp.split(')'))
tmp = tmp.replace('\r\n', ' ')

In [13]:
import json
results = json.loads(tmp)


In [14]:
results.keys()

dict_keys(['addr_count', 'place_totalcount', 'page_count', 'bus_cnt', 'busStop_cnt', 'bus_recommend', 'bus_recommend_top', 'query', 'org_query', 'guide_message', 'region_depth', 'region_type', 'sort', 'page', 'cate_id', 'mng_type', 'is_franchise', 'is_category', 'highway_yn', 'oil_yn', 'mrank_type', 'exposure_level', 'analysis', 'srcid', 'sn_query', 'samename', 'tile_search', 'target', 'trans_map_type', 'trans_map_str', 'bus', 'address_retry', 'address', 'premium', 'place', 'cateLink', 'category_depth', 'cateGroupList', 'busStop'])

In [15]:
print(results.get('query'), results.get('analysis'))

스타벅스 스타벅스|I10080304||||0
